#**Машинное обучение ИБ-2024**

#**Домашнее задание 2.**
#Классификация, KNN, LogReg, SVC.

In [1]:
from typing import Tuple, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

sns.set(style="darkgrid")

## **Теоретическая Часть**

Мы рассматриваем задачу бинарной классификации. Для прогнозирования мы хотели бы использовать модель логистической регрессии. Для регуляризации мы добавляем комбинацию штрафов в размере $l_2$ и $l_1$ (Elastic Net).

Каждый объект в обучающем наборе данных индексируется с помощью $i$ и описывается парой: объекты $x_i\in\mathbb{R}^{K}$ и двоичные метки $y_i$. Модель параметризуется со смещением $w_0\in\mathbb{R}$ и весами $w\in\mathbb{R}^K$.

Задача оптимизации в отношении $w_0, w$ заключается в следующем (Elastic Net Loss):

$$L(w, w_0) = \frac{1}{N} \sum_{i=1}^N \ln(1+\exp(-y_i(w^\top x_i+w_0))) + \gamma \|w\|_1 + \beta \|w\|_2^2$$.



Градиенты функции потерь логистической регрессии представлены ниже:

$$dL(w, w_0)/ dw = -\frac{1}{N}  \frac{X*y^\top}{1 + \exp(y (Xw+w_0)))} + \gamma * sign(w) + 2 * beta * w$$

$$dL(w, w_0)/ dw_0 = -\frac{1}{N}  \frac{y}{1 + \exp(y*(Xw+w_0)))}$$

#### 1. [0.5 Балл] Реализуйте функцию, выдающий значение функции потерь логичтической регрессии:

In [2]:
def loss(X, y, w: List[float], w0: float, gamma=1., beta=1.) -> float:
  """
  Функция потерь логистической регрессии.

  Args:
    X: Матрица признаков (N x D).
    y: Вектор меток (N x 1).
    w: Вектор весов (D x 1).
    w0: Смещение.
    gamma: Параметр регуляризации L1.
    beta: Параметр регуляризации L2.

  Returns:
    Значение функции потерь.
  """
  N = X.shape[0]
  y_pred = 1 / (1 + np.exp(-y * (X @ w + w0)))
  loss_val = -np.sum(np.log(y_pred)) / N
  loss_val += gamma * np.sum(np.abs(w)) + beta * np.sum(w**2)
  return loss_val
#  pass

#### 2. [0.5 Балл] Реализуйте функцию, которая будет возвращать градиенты весов вашей модели Логистической регрессии:

In [4]:
def get_grad(X, y, w: List[float], w0: float, gamma=1., beta=1.) -> Tuple[List[float], float]:
    '''
    :param X: np.ndarray of shape (n_objects, n_features) -- matrix objects-features
    :param y: np.ndarray of shape (n_objects,) -- vector of the correct answers
    :param w: np.ndarray of shape (n_feratures,) -- the weights
    :param w0: intercept
    :param gamma: penalty hyperparameter of L1-regularization
    :param beta: penalty hyperparameter of L2-regularization

    '''


    """
    Вычисляет градиенты весов и смещения для логистической регрессии.

    Args:
        X: Матрица признаков (N x D).
        y: Вектор меток (N x 1).
        w: Вектор весов (D x 1).
        w0: Смещение.
        gamma: Параметр регуляризации L1.
        beta: Параметр регуляризации L2.

    Returns:
        Кортеж, содержащий:
        - градиент весов (D x 1)
        - градиент смещения (1 x 1)
    """
    N = X.shape[0]
    y_pred = 1 / (1 + np.exp(-y * (X @ w + w0)))

    # Градиент весов
    grad_w = -np.sum((X.T * y * (1 - y_pred)) / N, axis=1) + gamma * np.sign(w) + 2 * beta * w
    grad_w = -np.sum((X.T * y * (1 - y_pred)) / N, axis=1) + gamma * np.sign(w) + 2 * beta * w


    # Градиент смещения
    grad_w0 = -np.sum(y * (1 - y_pred)) / N

    return grad_w.tolist(), grad_w0
#    pass

In [5]:
# код для проверки

np.random.seed(42)
X = np.random.multivariate_normal(np.arange(5), np.eye(5), size=10)
y = np.random.binomial(1, 0.42, size=10)
w, w0 = np.random.normal(size=5), np.random.normal()

grad_w, grad_w0 = get_grad(X, y, w, w0)
assert(np.allclose(grad_w,
                   [-2.73262076, -1.87176281, 1.30051144, 2.53598941, -2.71198109],
                   rtol=1e-2) & \
       np.allclose(grad_w0,
                   -0.2078231418067844,
                   rtol=1e-2)
)

####  3. [1 Балл]  Реализуйте класс для модели логистической регрессии, используя выше написанные функции:

Модель должна обучаться методом SGD.

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import roc_curve

In [ ]:
class Logit(BaseEstimator, ClassifierMixin):
    def __init__(self, beta=1.0, gamma=1.0, lr=1e-2, tolerance=1e-8, max_iter=1000, random_state=42):
        '''
        betta: penalty hyperparameter of L2-regularization
        gamma: penalty hyperparameter of L1-regularization
        tolerance: minimal allowed movement in each iteration
        lr: determines the step size at each iteration
        max_iter: maximum number of iterations taken for the solvers to converge

        '''
        """
        Инициализирует модель логистической регрессии.

        Args:
            beta: Параметр регуляризации L2.
            gamma: Параметр регуляризации L1.
            lr: Шаг обучения.
            tolerance: Порог для остановки градиентного спуска.
            max_iter: Максимальное количество итераций.
            random_state: Случайное начальное значение.
        """
        self.beta = beta
        self.gamma = gamma
        self.lr = lr
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.random_state = random_state
        self.w = None
        self.w0 = None


    def fit(self, X, y):
        """
        Обучает модель логистической регрессии.

        Args:
            X: Матрица признаков (N x D).
            y: Вектор меток (N x 1).
        """
        np.random.seed(self.random_state)
        self.w = np.random.normal(size=X.shape[1])
        self.w0 = np.random.normal()

        for _ in range(self.max_iter):
            grad_w, grad_w0 = get_grad(X, y, self.w, self.w0, self.gamma, self.beta)

            # Обновление весов
            self.w -= self.lr * grad_w
            self.w0 -= self.lr * grad_w0

            # Проверка на сходимость
            if np.linalg.norm(grad_w) < self.tolerance and np.abs(grad_w0) < self.tolerance:
                break

        return self


    def predict(self, X):
        """
        Function that returns the vector of predicted labels for each object from X
        Предсказывает метки для новых объектов.

        Args:
            X: Матрица признаков (N x D).

        Returns:
            Вектор предсказанных меток (N x 1).
        """
        y_pred = 1 / (1 + np.exp(-(X @ self.w + self.w0)))
        return (y_pred > 0.5).astype(int)


    def predict_proba(self, X):
        """
        Function that estimates probabilitie
        Предсказывает вероятности принадлежности к классу.

        Args:
            X: Матрица признаков (N x D).

        Returns:
            Вектор предсказанных вероятностей (N x 1).
        """
        y_pred = 1 / (1 + np.exp(-(X @ self.w + self.w0)))
        return y_pred

#        pass


In [ ]:
# этот код менять не надо!
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1800, n_features=2, n_redundant=0, n_informative=2,
                               random_state=42, n_clusters_per_class=1)

####  4. [0.5 Балл]  Реализуйте функцию, которая отрисовывает объекты вашего датасета, их метки и разделяющую гиперплоскость, полученную от Логистической регрессии (пример того, что должно получиться ниже):

In [ ]:
def plot_decision_boundary(model, X, y):
    """
    Отрисовывает объекты, их метки и разделяющую гиперплоскость.

    Args:
        model: Обученная модель логистической регрессии.
        X: Матрица признаков (N x D).
        y: Вектор меток (N x 1).
    """
    # Проверка, что модель обучена
    if model.w is None or model.w0 is None:
        raise ValueError("Модель должна быть обучена перед отрисовкой границы решения.")

    # Отрисовка объектов
    plt.scatter(X[y == 0, 0], X[y == 0, 1], c='blue', label='Класс 0')
    plt.scatter(X[y == 1, 0], X[y == 1, 1], c='red', label='Класс 1')

    # Вычисление координат разделяющей гиперплоскости
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))
    Z = 1 / (1 + np.exp(-(xx * model.w[0] + yy * model.w[1] + model.w0)))
    Z = Z > 0.5

    # Отрисовка разделяющей гиперплоскости
    plt.contour(xx, yy, Z, colors='green', linewidths=2)

    # Настройка графика
    plt.xlabel('Признак 1')
    plt.ylabel('Признак 2')
    plt.title('Разделяющая гиперплоскость')
    plt.legend()
    plt.show()
#    pass

In [ ]:
model = Logit(0,0)
y[y == 0] = -1
model.fit(X, y)
plot_decision_boundary(model, X, y)

TypeError: can't multiply sequence by non-int of type 'float'

#### 5. [0.5 Балл] Для предыдущей задачи отобразите на графике, как изменяется значение функция потерь от номера итерации.

In [ ]:
def plot_loss_history(model):
    pass

In [ ]:
plot_loss_history(model)

#### 6. [2 Балл] Для данных, на которых тестировали модель Логистической регрессии, заиспользуйте модель SVC из библиотеки sklearn. Попробуйте различные ядра (kernel) и различные коэфициенты C. Посмотрите на метрики, которые мы обсуждали на занятии (Acc, Precision, Recall, AUC-ROC, F1-Score).

#### 7. [2 Балл] Реализуйте класс KNNClassifier, который должен реализовывать классификацию путем нахождения k ближайших соседей. В методе predict_proba Вам необходимо выдавать вектор вероятностей для каждого объекта, который означает, что объект является экземпляром i-го класса с p_i вероятностью. Протестируйте Ваш класс на данных, сгенерированных выше, посмотрите на метрики (Acc, Precision, Recall, AUC-ROC, F1-Score).

In [ ]:
class KNNClassifier:
    def __init__(self, n_neighbors=5, metric='euclidean'):
      ...

    def fit(self, X, y):
      ...

    def predict(self, X):
      ...

    def predict_proba(self, X):
      ...

## **Практическая часть**

В этом задании мы будем работать с Датасетом Fashion Mnist. Это датасет, который представляет изображения одного канала с различными типами одежды. Вам необходимо провести полный пайплайн обучения моделей (KNN и Logreg), которые вы можете импортировать из библиотеки sklearn.

#### 8. [0 Балл] Импортируйте датафрейм из csv файла. Поделите выборку следующим образом - :50000 (Train) и 50000: (Test).

#### 9. [0.5 Балл] Визуализируйте некоторые из объектов датасета. В колонках отображены яркости пикселей, которые представляют из себя изображения Fashion Mnist. С помощью matplotlib визуализируйте по одному представителю каждого класса.

#### 10. [0.5 Балл] Отнормируйте признаки в датасете, попробуйте два варианта StandartScaller и MinMaxScaller.

#### 10. [2 Балл] Проведите эксперименты: для моделей KNeighborsClassifier и LogisticRegression подберите гиперпараметры с помощью GridSerchCV (минимум 5 фолдов). Получите качество моделей на тестовой выборке. Основная метрика в данном задании будет accuracy. Сравните эти две модели. Какая модель показывает лучшее качество, предположите почему и напишите ответ.

**NB!**: в задании нужно подбирать несколько гиперпараметров по сетке. Какие гиперпараметры подбирать - решаете Вы сами. Обязательно обоснуйте, почему и какие параметры Вы подбираете! Например, подбор только гиперпараметра C в LogisticRegression не будет засчитываться как решение данного задания! Попытайтесь серьезно отнестись к нему, будто вы за это получите зарплату 300к.

## **Бонусы**

#### Задача 1. [1 Балл] У Вас есть датасет с 10**4 объектами. У всех объектов два признака и все они одинаковые у всех объектов. Однако, 5000 - отрицательного класса и 5000 - положительного класса. Вы запускате Логистическую регрессию для классификации на данном датасете. Что Вы получите в итоге обучения данной модели на SGD? Ответ обоснуйте.

#### Задача 2. [1 Балл] При классификации Fashion Mnist модель Логистической регрессии на обучении многоклассовой классификации методом One-VS-All у Вас получилось k классификаторов. Изобразите веса ваших полученных моделей как изображения в matplotlib. Возможно, модель выучила какие-то графические паттерны в данных? Ответ обоснуйте.

#### Задача 3. [1 Балл] В задаче классификации Fashion Mnist Вы попытались выбить какой-то accuracy. Для получения бонусного балла Вам нужно на той же самой выборке получить значение метрики accuracy > 0.87 на тесте (Тестовую выборку менять нельзя, но обучающую можно). Какими моделями и методами Вы это будете делать - на Ваше усмотрение, но **нельзя использовать никакие нейронные сети**. Необходимо получить модель машинного обучения, выполняющую эту задачу.